In [160]:
import pandas as pd

# Load the Excel file into a DataFrame
df_bridges_all = pd.read_excel("bridges_data_p2.xlsx")

In [161]:
import geopandas as gpd
# Replace the file path with your actual file path
# region_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
region_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
region_gdf = gpd.read_file(region_path)


# districts_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # doma
districts_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
districts_gdf = gpd.read_file(districts_path)


# Replace the file path with your actual file path
# file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # doma
file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # doma


# Load the GeoJSON file into a GeoDataFrame
settle_gdf = gpd.read_file(file_path)

# Define the types to exclude
types_to_exclude = ['country', 'airport', 'Psie skaly',
                    'county', 'Drevenný kostol', 'Vodopád']

# Create a boolean mask to select rows where the 'type' column is not in types_to_exclude
# and the 'population' column is not null
mask = (~settle_gdf['type'].isin(types_to_exclude)) & (
    settle_gdf['population'] > 0)


# Apply the mask to filter the GeoDataFrame in place
settle_gdf = settle_gdf[mask]

# Drop rows where the 'name' column is "Bratislava" or "Košice" from settle_gdf
settle_gdf = settle_gdf[~settle_gdf["name"].isin(["Bratislava", "Košice"])]

In [162]:
import numpy as np

# Define a function to select a random bridge from each group


def random_bridge(group):
    # Select one random sample from each group
    return group.sample(4, random_state=420)


# Group by "Kraj" and apply the random_bridge function
random_bridges = df_bridges_all.groupby(
    "Kraj", group_keys=False).apply(random_bridge)

# Reset index to make it a DataFrame
random_bridges = random_bridges.reset_index(drop=True)

# Display the randomly selected bridges
random_bridges

,ID_mosta,Trieda_PK,Číslo_PK,Názov_mosta,správcovské_číslo,Zemepisná_dĺžka,Zemepisná_šírka,Výška,Druh_konštrukcie,Material,...,n_2016,n_2017,n_2018,n_2019,n_2020,n_2021,n_2022,n_2023,Trieda_PK_n,Okres_n
0,M35,cesta III. triedy,2749,2749-1 (050126-001) Most cez potok Pápča v obc...,1,20.064702,48.468025,227.524500,dosková,monolitický železobetón,...,2,2,2,2,2,4,4,4,4,609
1,M925,cesta I. triedy,65,"65_035 Most cez Záhrbský potok, k.ú. Nová Baňa",35,18.633559,48.409204,203.823999,dosková,monolitický železobetón,...,3,3,3,3,3,4,4,5,2,612
2,M5982,cesta III. triedy,2418,2418-2_Most cez Selčiansky potok v B. Bystrici...,2,19.187009,48.746536,361.146999,dosková,monolitický železobetón,...,3,3,3,3,3,3,3,4,4,601
3,M9611,diaľnica,R2,Most na R2 nad Slatinou,116,19.363950,48.540930,362.000000,dosková,monolitický predpätý betón,...,0,0,0,0,0,0,2,2,1,604
4,M4498,diaľnica,D1,Most na vetve 3 križovatky Gagarinova,023A4,17.181763,48.151013,186.041249,Nezistený,Nezistený,...,0,0,0,0,0,0,2,2,1,102
5,M7029,cesta II. triedy,505,"Most cez potok Mláka v osade Poľný Mlyn, Devín...",505-005,16.997337,48.247369,200.398499,dosková,monolitický železobetón,...,5,5,5,5,5,5,4,4,3,104
6,M2583,diaľnica,D2,Most cez Bystrický potok,109,17.017404,48.238301,154.850662,trámová,prefabrikovaný železobetón,...,4,4,4,4,4,4,4,4,1,104
7,M5964,cesta III. triedy,1062,III_503007 - 004 Most cez Malý Dunaj - Tomášov,4,17.342084,48.152760,129.007500,rámová,monolitický predpätý betón,...,4,4,4,4,4,4,4,4,4,108
8,M5854,cesta III. triedy,3799,Mosty cez potok Slaný pred obcou Baškovce,1,22.201606,48.774259,153.332750,dosková,prefabrikovaný železobetón,...,3,3,3,3,3,3,3,6,4,809
9,M7232,cesta II. triedy,566,Most cez potok Syrový za obcou Choňkovce,2,22.248135,48.784600,213.763999,dosková,prefabrikovaný predpätý betón,...,4,4,6,6,6,6,6,6,3,809


In [163]:
# from geopy.distance import geodesic
# import pandas as pd
# from shapely.geometry import Point


# def find_nearby_settlements(bridge_row, settlements_gdf, radius_km=[2, 7, 17]):
#     # Create a point geometry for the bridge
#     bridge_point = (bridge_row['Zemepisná_šírka'],
#                     bridge_row['Zemepisná_dĺžka'])

#     # Initialize dictionaries to hold the settlements' details within each distance range
#     settlements_within_1km = {}
#     settlements_within_1_to_5km = {}
#     settlements_within_5_to_15km = {}

#     # Calculate the distance from the bridge to each settlement in kilometers and their population
#     for _, settle_row in settlements_gdf.iterrows():
#         # Extract the point geometry for the settlement
#         # Assuming geometry is a Point
#         settle_point = (settle_row['geometry'].y, settle_row['geometry'].x)

#         # Calculate the geodesic distance
#         distance_km = geodesic(bridge_point, settle_point).kilometers

#         # Prepare the settlement details including distance and population
#         settlement_details = {
#             'distance_km': distance_km,
#             'population': settle_row['population']
#         }

#         # Categorize the settlement based on the distance and add it to the respective dictionary
#         if distance_km <= radius_km[0]:
#             settlements_within_1km[settle_row['name']] = settlement_details
#         elif radius_km[0] < distance_km <= radius_km[1]:
#             settlements_within_1_to_5km[settle_row['name']
#                                         ] = settlement_details
#         elif radius_km[1] < distance_km <= radius_km[2]:
#             settlements_within_5_to_15km[settle_row['name']
#                                          ] = settlement_details

#     # Return a Series with dictionaries of settlements for each distance range
#     return pd.Series([settlements_within_1km, settlements_within_1_to_5km, settlements_within_5_to_15km],
#                      index=['within_1km', 'within_1_to_5km', 'within_5_to_15km'])

In [164]:
# # Apply the function to each bridge
# nearby_settlements = random_bridges.apply(
#     lambda row: find_nearby_settlements(row, settle_gdf), axis=1)

# # Concatenate the results with the original random_bridges dataframe
# random_bridges_with_settlements = pd.concat(
#     [random_bridges, nearby_settlements], axis=1)

In [165]:
# cislo = 0
# random_bridges_with_settlements.iloc[cislo]["Kraj"], random_bridges_with_settlements.iloc[cislo]["within_1km"], random_bridges_with_settlements.iloc[
#     cislo]["within_1_to_5km"], random_bridges_with_settlements.iloc[cislo]["within_5_to_15km"]

In [166]:
from geopy.distance import geodesic
import pandas as pd
from shapely.geometry import Point


def find_population_sums(bridge_row, settlements_gdf, radius_km=[2, 7, 17]):
    # Create a point geometry for the bridge
    bridge_point = (bridge_row['Zemepisná_šírka'],
                    bridge_row['Zemepisná_dĺžka'])

    # Initialize counters for the total population within each distance range
    population_within_1km = 0
    population_within_1_to_5km = 0
    population_within_5_to_15km = 0

    # Calculate the distance from the bridge to each settlement in kilometers and accumulate population
    for _, settle_row in settlements_gdf.iterrows():
        # Extract the point geometry for the settlement
        settle_point = (settle_row['geometry'].y, settle_row['geometry'].x)

        # Calculate the geodesic distance
        distance_km = geodesic(bridge_point, settle_point).kilometers

        # Accumulate the population based on the distance
        if distance_km <= radius_km[0]:
            population_within_1km += settle_row['population']
        elif radius_km[0] < distance_km <= radius_km[1]:
            population_within_1_to_5km += settle_row['population']
        elif radius_km[1] < distance_km <= radius_km[2]:
            population_within_5_to_15km += settle_row['population']

    # Return a Series with the total population sums for each distance range
    return pd.Series([population_within_1km, population_within_1_to_5km, population_within_5_to_15km],
                     index=['population_1ST', 'population_2ND', 'population_3RD'])

In [167]:
# # Apply the function to each bridge to get the population sums within specified distance ranges
# population_sums = df_bridges_all.apply(
#     lambda row: find_population_sums(row, settle_gdf), axis=1)


In [168]:
import pandas as pd


def find_population_sums_with_progress(index, row, settlements_gdf, total_rows):
    if index % 50 == 0:
        print(f"Processing row {index + 1} of {total_rows}...")
    return find_population_sums(row, settlements_gdf)


# Total number of rows for progress tracking
total_rows = len(df_bridges_all)

# Apply the function to each bridge to get the population sums within specified distance ranges, with progress tracking
population_sums = df_bridges_all.apply(
    lambda row: find_population_sums_with_progress(row.name, row, settle_gdf, total_rows), axis=1)


print("Completed processing all bridges.")

Processing row 1 of 8125...
Processing row 51 of 8125...
Processing row 101 of 8125...
Processing row 151 of 8125...
Processing row 201 of 8125...
Processing row 251 of 8125...
Processing row 301 of 8125...
Processing row 351 of 8125...
Processing row 401 of 8125...
Processing row 451 of 8125...
Processing row 501 of 8125...
Processing row 551 of 8125...
Processing row 601 of 8125...
Processing row 651 of 8125...
Processing row 701 of 8125...
Processing row 751 of 8125...
Processing row 801 of 8125...
Processing row 851 of 8125...
Processing row 901 of 8125...
Processing row 951 of 8125...
Processing row 1001 of 8125...
Processing row 1051 of 8125...
Processing row 1101 of 8125...
Processing row 1151 of 8125...
Processing row 1201 of 8125...
Processing row 1251 of 8125...
Processing row 1301 of 8125...
Processing row 1351 of 8125...
Processing row 1401 of 8125...
Processing row 1451 of 8125...
Processing row 1501 of 8125...
Processing row 1551 of 8125...
Processing row 1601 of 8125...


In [169]:
# Concatenate the results with the original random_bridges dataframe
df_bridges_all_with_population_sums = pd.concat(
    [df_bridges_all, population_sums], axis=1)

In [170]:
df_bridges_all_with_population_sums

,ID_mosta,Trieda_PK,Číslo_PK,Názov_mosta,správcovské_číslo,Zemepisná_dĺžka,Zemepisná_šírka,Výška,Druh_konštrukcie,Material,...,n_2019,n_2020,n_2021,n_2022,n_2023,Trieda_PK_n,Okres_n,population_1ST,population_2ND,population_3RD
0,M5897,cesta I. triedy,2,02_002 Most cez potok Brestovec v obci Kopčany,2,17.123319,48.790152,163.816250,dosková,prefabrikovaný železobetón,...,5,5,5,5,5,2,206,0,12413,28874
1,M1979,cesta I. triedy,2,02_003 Most cez Cuninský potok medzi obcami Ko...,3,17.079098,48.757287,162.974249,dosková,prefabrikovaný predpätý betón,...,6,6,6,6,6,2,206,0,5149,41725
2,M6525,cesta I. triedy,2,02_004 Most nad železnicou Praha - Bratislava ...,4,17.031675,48.684515,168.968750,dosková,prefabrikovaný predpätý betón,...,4,4,4,4,4,2,206,0,6182,41894
3,M4584,cesta I. triedy,2,O2_005 Most cez Zelnický potok pred obcou Kúty,5,17.020574,48.663688,155.871499,dosková,monolitický železobetón,...,3,3,3,3,4,2,205,4121,3623,27448
4,M3053,cesta I. triedy,2,"02_006 Most nad diaľnicou D2, pri obci Kúty",6,17.001660,48.639120,161.480000,trámová,monolitický predpätý betón,...,4,4,5,5,5,2,205,0,11455,27162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8120,M9963,diaľnica,R2,"Most nad PC v km 9,000",SO 212-0,19.533000,48.474460,271.742065,rámová,monolitický prostý betón,...,0,0,0,0,2,1,606,1203,6601,46474
8121,M9965,diaľnica,R2,"Most na R2 nad III/2632 km 10,500",SO 214,19.547370,48.465050,262.075714,dosková,monolitický predpätý betón,...,0,0,0,0,2,1,606,3539,4265,46474
8122,M9966,diaľnica,R2,"Most na R2 v km 11,450",SO 215,19.559300,48.462190,255.026825,trámová,monolitický predpätý betón,...,0,0,0,0,2,1,606,1486,9244,71525
8123,M9967,diaľnica,R2,"Most na R2 nad PC v km 14,500",SO 216,19.579420,48.438690,244.832169,dosková,monolitický železobetón,...,0,0,0,0,2,1,606,2073,8555,62027


In [171]:
# Save the DataFrame as an Excel file
df_bridges_all_with_population_sums.to_excel(
    "bridges_data_p3.xlsx", index=False)